<a href="https://colab.research.google.com/github/antonercool/deep-learning-exam-project/blob/master/FallDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning course project - Fall Detection




###Group members


*   Anton Sihm, studentId.: 201504954
*   Peter Marcus Hoveling, studentId.: 201508876







###Purpose

This project aims to detect human actions from live video feed, in order to achieve safety, such as
alarms when elderly people fall in their apartments.








---



### Data sets

To evaluate the fall detection, dataset from realistic video surveillance settings will be used. Framerate
and resolution may vary as finding the same formatted video data is challenging


*   mViA - Fall detection dataset



## Task 0: Mounting Google Drive data


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Creating a variable to the specific files mouted

In [2]:
from pathlib import Path
root = '/content/gdrive/My Drive/' # Don't change this
data_dirname = 'Data_complete' # Change as you like
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) # should "/content/gdrive/My Drive/Data"

Validating the data directory

In [3]:
print(p)

/content/gdrive/My Drive/Data_complete


## Task 1: Preprocessing

## Utils

In [15]:
# Importing all necessary libraries
import cv2
import os
import numpy 
import random

class VideoUtils:

  def __init__(self):
    pass

  def convert_single_video_to_images(self, video_path, annotation_path, dim):
    # Read the video from specified path
    video = cv2.VideoCapture(video_path)
    meta_data = open(annotation_path)

    start_fall_frame = meta_data.readline() 
    end_fall_frame = meta_data.readline() 

    not_falling = False
    # check if has labels
    if "," in start_fall_frame:
      #print(f"does not contain : {annotation_path}")
      ## skip current
      not_falling = True
      start_fall_frame = 0
      end_fall_frame = 0
    else:
      start_fall_frame = int(start_fall_frame)
      end_fall_frame = int(end_fall_frame)  

    traning_samples = []
    training_labels = []

    currentframe = 0

    while(True):

      # reading from frame
      ret,frame = video.read()
      if ret:
        if(frame.shape[0] != dim[0] or frame.shape[1] != dim[1]):
          try:
            frame = self.center_crop(frame, dim)
          except e:
            print(f"failed to center from image from vid : {video_path}, error {e}")

        traning_samples.append(frame)

        if not_falling == False:
          # if falling - label 1
          if currentframe >= start_fall_frame and currentframe <= end_fall_frame:
            training_labels.append(1)
          # not falling - label 0
          else:
            training_labels.append(0)
        else:
          training_labels.append(0)

        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
      else:
        break

    # Release all space and windows once done
    video.release()
    cv2.destroyAllWindows()

    traning_samples = numpy.array(traning_samples)
    training_labels =  numpy.array(training_labels)

    return (traning_samples, training_labels, True)

  # Function from: https://medium.com/curious-manava/center-crop-and-scaling-in-opencv-using-python-279c1bb77c74
  def center_crop(self, img, dim):
	  """Returns center cropped image
	  Args:
	  img: image to be center cropped
	  dim: dimensions (width, height) to be cropped
	  """
	  width, height = img.shape[1], img.shape[0]

	  # process crop width and height for max available dimension
	  crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	  crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	  mid_x, mid_y = int(width/2), int(height/2)
	  cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	  crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
	  return crop_img


  def _get_movie_IDs(self, root_path):
    list_video_paths = []
    annotation_path = r"/Annotation_files"
    video_path = r"/Videos"
    
    data_sub_dir = os.listdir(root_path)

    # interate each sub dir
    for dir in data_sub_dir:
      if ".ipynb" in dir:
        continue
      # for each file in sub dir
      for file in os.listdir(str(root_path) + "/" + dir + video_path):
        absolute_video_file_path = str(str(root_path) + "/" + dir + video_path + "/"+ file)
        list_video_paths.append(absolute_video_file_path)

    return list_video_paths


  # create_partition_list(p)
  def create_partition_list(self, root_path, traning_percentage=80, validation_percentage=20):
    partition = {}

    absolute_video_path_list = video_utils._get_movie_IDs(root_path)
    random.shuffle(absolute_video_path_list)

    to_index = int((len(absolute_video_path_list)+1)*(traning_percentage/100))
    from_index = int(len(absolute_video_path_list)*(traning_percentage/100)+1)

    print(f"traning {0}:{to_index}")
    print(f"validation {from_index}:{len(absolute_video_path_list)}")

    training = absolute_video_path_list[0:to_index]
    validation = absolute_video_path_list[from_index:]

    total_frames_training = 0
    total_frames_validation = 0
    
    for image_path in training:
      cap= cv2.VideoCapture(image_path)
      total_frames_training += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for image_path in validation:
      cap= cv2.VideoCapture(image_path)
      total_frames_validation += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    
    partition['Training'] = (training, total_frames_training)
    partition['Validation'] = (validation, total_frames_validation)

    return partition

### Testing Video ultils


In [5]:
video_utils = VideoUtils()
traning_samples, training_labels, is_ok = video_utils.convert_single_video_to_images(str(p) + "/Office/Videos/video (6).avi", str(p) + "/Office/Annotation_files/video (6).txt")
print(traning_samples.shape)

absolute_video_path_list = video_utils._get_movie_IDs(p)

partition = video_utils.create_partition_list(p)

print(partition['Training'][1])
print(partition['Validation'][1])


TypeError: ignored

### Dataloader

In [491]:
import numpy as np
import keras
from tensorflow.keras.utils import Sequence
from keras import utils as np_utils
import random

class FallingDataGenerator(Sequence):

  def __init__(self, movie_path_list, total_movie_frames, batch_size=32, dim=(240, 320), n_channels=3, n_classes=2, shuffle=True): 
      'Initialization'
      self.dim = dim
      self.batch_size = batch_size
      self.n_channels = n_channels
      self.n_classes = n_classes
      self.shuffle = shuffle
      self.movie_path_list = movie_path_list
      self.total_movie_frames = total_movie_frames
      self.current_video_data = 0# ((415, 240, 320, 3)(415)) # 0-32, 32-64 ...... 400-415    
      self.current_video_frame = 0
      self.current_movie_index = 0
      self.video_utils = VideoUtils()
      self.current_batch = 0

      # init by shuffle
      self.on_epoch_end()

  # Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike
  ## TODO shuffle path list
  def on_epoch_end(self):
    'Updates indexes after each epoch'
    print("DEBUG - on epoch end")
    random.shuffle(self.movie_path_list)
    self.current_video_data = 0
    self.current_video_frame = 0
    self.current_movie_index = 0


  def get_indexes(self):
    return self.indexes

  def __data_generation(self):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.zeros((self.batch_size, *self.dim, self.n_channels)) # (32, H, W, 3)
    Y = np.zeros((self.batch_size), dtype=int)

    ## Init - first time load
    if(self.current_video_data == 0 and self.current_video_frame == 0 and self.current_movie_index == 0):
      #####print("DEBUG - First time load")
      video_path = self.movie_path_list[self.current_movie_index]
      annotation_path = video_path.replace("Videos", "Annotation_files")
      annotation_path = annotation_path.replace(".avi", ".txt")

      self.current_video_data = self.video_utils.convert_single_video_to_images(video_path, annotation_path, self.dim)
    
    total_number_of_current_images = len(self.current_video_data[0])
    #####print(f"DEBUG - total_number_of_current_images : {total_number_of_current_images}")

    # Check if dont have enough frames
    if(self.current_video_frame + self.batch_size + 1 > total_number_of_current_images):
      #####print(f"DEBUG - not have enough frames{self.current_video_frame + self.batch_size -1}>{total_number_of_current_images} - taking from current and current + 1")
      # How many frames did we overextend with
      residual_value_from_current_video = (self.current_video_frame + self.batch_size)%total_number_of_current_images
      # How many frames are we missing to complete current video
      frames_missing = self.batch_size - residual_value_from_current_video

      # loop for the remaining needed frames
      for i in range(frames_missing):
        #####print(f"DEBUG - taking frame_index: {self.current_video_frame}")
        X[i,] = self.current_video_data[0][self.current_video_frame]
        Y[i,] = self.current_video_data[1][self.current_video_frame]
        self.current_video_frame += 1
    
      # Continue to next video
      self.current_movie_index += 1
      self.current_video_frame = 0
      self.current_video_data = 0

      # If we have more movies
      # 0, 1, 2     -- 2<=2
      if(self.current_movie_index <= (len(self.movie_path_list) - 1)):
        #####print("DEBUG - taking new vid")
        video_path = self.movie_path_list[self.current_movie_index]
        annotation_path = video_path.replace("Videos", "Annotation_files")
        annotation_path = annotation_path.replace(".avi", ".txt")

        self.current_video_data = self.video_utils.convert_single_video_to_images(video_path, annotation_path, self.dim)
        for i in range(frames_missing, self.batch_size):
          X[i,] = self.current_video_data[0][self.current_video_frame]
          Y[i,] = self.current_video_data[1][self.current_video_frame]
          self.current_video_frame += 1
        
    # We do have enough frames
    else:
      #####print("DEBUG - We have frames - taking from current only")
      for i in range(self.batch_size):
        X[i,] = self.current_video_data[0][self.current_video_frame]
        Y[i,] = self.current_video_data[1][self.current_video_frame]
        self.current_video_frame += 1
      
      # If we have an exact match of video frames and have more videos
      #####print(f"DEBUG {self.current_video_frame} == {total_number_of_current_images} and {self.current_movie_index} <= {(len(self.movie_path_list) - 1)}")
      if(self.current_video_frame == total_number_of_current_images and self.current_movie_index <= (len(self.movie_path_list) - 1)):
        #####print(f"DEBUG - having exact match")
        # Continue to next video
        self.current_movie_index += 1
        self.current_video_frame = 0

        video_path = self.movie_path_list[self.current_movie_index]
        annotation_path = video_path.replace("Videos", "Annotation_files")
        annotation_path = annotation_path.replace(".avi", ".txt")

        self.current_video_data = self.video_utils.convert_single_video_to_images(video_path, annotation_path, self.dim)
    
    return X, keras.utils.np_utils.to_categorical(Y, num_classes=self.n_classes)

  def __len__(self):
    'Denotes the number of batches per epoch'
    ## Total number of frames in all videos / batchsize
    return int(np.floor(self.total_movie_frames / self.batch_size))


  def __getitem__(self):
    'Generate one batch of data'
    
    if(self.current_batch > self.__len__()-1):
      self.on_epoch_end()

    # Generate data
    X, y = self.__data_generation()
    
    return X, y


### Testing data loader


In [13]:
#utils_video
video_utils = VideoUtils()

# Datasets
partition = video_utils.create_partition_list(p)

traning 0:152
validation 153:190


In [492]:
# Parameters
params = {'batch_size': 32,
          'dim': (180,180),
          'n_channels': 3,
          'n_classes': 2, 
          'shuffle': True}

# Generators
short = partition['Training'][0][0:3]
training_generator = FallingDataGenerator(short, partition['Training'][1], **params)
validation_generator = FallingDataGenerator(partition['Validation'][0], partition['Validation'][1], **params)

DEBUG - on epoch end
DEBUG - on epoch end


In [521]:
x,y = training_generator.__getitem__()
print(f"current movie file index : {training_generator.current_movie_index}")
print(f"current video frame in movie: {training_generator.current_video_frame}")
print(f"batch : {len(x)}")

1858
DEBUG - total_number_of_current_images : 467
DEBUG - We have frames - taking from current only
DEBUG 434 == 467 and 1 <= 2
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
current movie file index : 1
current video frame in movie: 434
batch : 32
